<a href="https://colab.research.google.com/github/ole12345/yolov9/blob/main/yolo_v9train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/ole12345/yolov9.git

Cloning into 'yolov9'...
remote: Enumerating objects: 532, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 532 (delta 11), reused 23 (delta 7), pack-reused 499 (from 1)
Receiving objects: 100% (532/532), 2.62 MiB | 18.13 MiB/s, done.
Resolving deltas: 100% (286/286), done.


In [ ]:
%cd yolov9
!pip install -r requirements.txt --quiet
import os
os.kill(os.getpid(), 9)
